In [1]:
#!pip install georasters
#!pip install geoindex
#!pip install spark
#!pip install pyspark

In [2]:
import pandas as pd
import georasters as gr
import matplotlib.pyplot as plt
from geoindex import GeoGridIndex, GeoPoint
import os
from tqdm import tqdm, tqdm_pandas

In [3]:
pd.set_option('display.max_columns', 100)

## Load Water Data

In [4]:
os.getcwd()

'/Users/alexjamar/Documents/Berkeley/Capstone/Capstone_Water_Access/Population'

In [5]:
water_data = pd.read_csv("/Users/alexjamar/Documents/Berkeley/Capstone/Capstone_Water_Access//data/Water_Point_Data_Exchange_Complete_Dataset.csv")

/Users/alexjamar/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2,5,7,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
water_data.head()

,Row ID,#country_name,#water_source,#water_tech,#status_id,#management,#pay,#installer,#install_year,#status,#source,#adm1,#adm2,#wpdx_id,#report_date,#country_id,#activity_id,#data_lnk,#orig_lnk,photo_lnk,#converted,#created,#updated,#lat_deg,#lon_deg,Location,Count,#fecal_coliform_presence,#fecal_coliform_value,#subjective_quality
0,1,"Tanzania, United Republic of",Spring,Gravity Communal standpipe,no,NaN,Never pay,RWE,1972.0,Status:Not functional|Quantity:Dry|Quality:Soft,SNV,Arusha,Longido,wpdx-00000001,01/05/2008 12:00:00 AM +0000,TZ,NaN,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,NaN,NaN,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.83257,36.3090,"(-2.83257, 36.309)",1,NaN,NaN,NaN
1,2,"Tanzania, United Republic of",Rainwater harvesting,Gravity Communal standpipe,yes,NaN,Never pay,World Vision,2005.0,Status:Functional|Breakdown Year:2007|Reason N...,SNV,Arusha,Longido,wpdx-00000002,01/01/2008 12:00:00 AM +0000,TZ,NaN,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,NaN,NaN,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.84781,36.5284,"(-2.84781, 36.5284)",1,NaN,NaN,NaN
2,3,"Tanzania, United Republic of",Rainwater harvesting,Gravity Communal standpipe,yes,NaN,Never pay,World Vision,2005.0,Status:Functional|Quantity:Seasonal|Quality:Soft,SNV,Arusha,Longido,wpdx-00000003,01/01/2008 12:00:00 AM +0000,TZ,NaN,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,NaN,NaN,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.84796,36.5288,"(-2.84796, 36.5288)",1,NaN,NaN,NaN
3,4,"Tanzania, United Republic of",Rainwater harvesting,Gravity Communal standpipe,yes,NaN,Never pay,World Vision,2005.0,Status:Functional|Quantity:Seasonal|Quality:Soft,SNV,Arusha,Longido,wpdx-00000004,01/01/2008 12:00:00 AM +0000,TZ,NaN,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,NaN,NaN,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.85318,36.5276,"(-2.85318, 36.5276)",1,NaN,NaN,NaN
4,5,Nicaragua,NaN,NaN,yes,NaN,NaN,NaN,2011.0,NaN,El Porvenier,Matagalpa,Terrabona,wpdx-00000005,01/01/2011 12:00:00 AM +0000,NI,NaN,http://www.dropbox.com/s/0f70mln5roxhy29/ProjT...,https://www.waterpointdata.org/dataset/resourc...,NaN,"#data_lnk, #lon_deg, #lat_deg, #country_id",04/08/2016 08:35:49 AM +0000,04/08/2016 08:35:49 AM +0000,12.74070,-86.0121,"(12.7407, -86.0121)",1,NaN,NaN,NaN


In [7]:
#Create df with only Swaziland water
sl_water = water_data[water_data['#country_name'] == 'Sierra Leone']

In [8]:
sl_water.head()

,Row ID,#country_name,#water_source,#water_tech,#status_id,#management,#pay,#installer,#install_year,#status,#source,#adm1,#adm2,#wpdx_id,#report_date,#country_id,#activity_id,#data_lnk,#orig_lnk,photo_lnk,#converted,#created,#updated,#lat_deg,#lon_deg,Location,Count,#fecal_coliform_presence,#fecal_coliform_value,#subjective_quality
42474,63115,Sierra Leone,Pump on hand-dug well,India Mark,yes,Community Management,Only if there is a breakdown,OXFAM,2011.0,Yes- functional.,Sierra Leone STATWASH portal,Eastern,Kailahun,wpdx-00063115,03/15/2012 12:00:00 AM +0000,SL,ae734jla,http://www.dropbox.com/s/8sgpa099ebtbg2w/All_W...,NaN,NaN,"#country_id, #status_id, #status, #management,...",10/26/2015 09:24:51 PM +0000,10/26/2015 09:24:51 PM +0000,8.14546,-10.7758,"(8.14546, -10.7758)",1,NaN,NaN,NaN
42475,63146,Sierra Leone,Pump on hand-dug well,Inkar,yes,Community Management,Yes- regularly,OXFAM,2010.0,Yes- functional.,Sierra Leone STATWASH portal,Eastern,Kailahun,wpdx-00063146,03/14/2012 12:00:00 AM +0000,SL,ahmcg6kg,http://www.dropbox.com/s/8sgpa099ebtbg2w/All_W...,NaN,NaN,"#country_id, #status_id, #status, #management,...",10/26/2015 09:24:51 PM +0000,10/26/2015 09:24:51 PM +0000,8.21998,-10.6945,"(8.21998, -10.6945)",1,NaN,NaN,NaN
42476,63116,Sierra Leone,Pump on hand-dug well,India Mark,yes,Community Management,No- water is free,Unknown,2011.0,Yes- functional.,Sierra Leone STATWASH portal,Eastern,Kailahun,wpdx-00063116,03/14/2012 12:00:00 AM +0000,SL,alqbuvky,http://www.dropbox.com/s/8sgpa099ebtbg2w/All_W...,NaN,NaN,"#country_id, #status_id, #status, #management,...",10/26/2015 09:24:51 PM +0000,10/26/2015 09:24:51 PM +0000,8.30947,-10.6979,"(8.30947, -10.6979)",1,NaN,NaN,NaN
42477,63117,Sierra Leone,Pump on hand-dug well,India Mark,yes,Community Management,Yes- regularly,OXFAM,2011.0,Yes- functional.,Sierra Leone STATWASH portal,Eastern,Kailahun,wpdx-00063117,03/08/2012 12:00:00 AM +0000,SL,aln4y85f,http://www.dropbox.com/s/8sgpa099ebtbg2w/All_W...,NaN,NaN,"#country_id, #status_id, #status, #management,...",10/26/2015 09:24:51 PM +0000,10/26/2015 09:24:51 PM +0000,8.30754,-10.7395,"(8.30754, -10.7395)",1,NaN,NaN,NaN
42478,63118,Sierra Leone,Pump on hand-dug well,India Mark,no,Community Management,No- water is free,cord sl,2011.0,No- broken down. Pump damaged,Sierra Leone STATWASH portal,Eastern,Kailahun,wpdx-00063118,03/09/2012 12:00:00 AM +0000,SL,alafpbam,http://www.dropbox.com/s/8sgpa099ebtbg2w/All_W...,NaN,NaN,"#country_id, #status_id, #status, #management,...",10/26/2015 09:24:51 PM +0000,10/26/2015 09:24:51 PM +0000,8.29985,-10.7054,"(8.29985, -10.7054)",1,NaN,NaN,NaN


## Radius-based Search

In [9]:
sl_pop = pd.read_csv("Sierra Leone_pop_data.csv")

In [10]:
#Create Geo Index of Sierra Leone Population Data
geo_index = GeoGridIndex()
for index, row in sl_pop.iterrows():
    geo_index.add_point(GeoPoint(row['y'], row['x'], ref=row['value']))

In [11]:
#Create Geo Index of Sierra Leone Water Point Data
geo_index_water = GeoGridIndex()
for index, row in sl_water.iterrows():
    try:
        geo_index_water.add_point(GeoPoint(row['#lat_deg'], row['#lon_deg'], ref=row['#status_id']))
    except:
        print("Row ID:", row['Row ID'])
        print("Lat:", row['#lat_deg'])
        print("Long:", row['#lon_deg'])

Row ID: 647413
Lat: 823769.0
Long: 135507.0
Row ID: 649398
Lat: 823867.0
Long: 135297.0
Row ID: 656955
Lat: 821575.0
Long: 134656.0
Row ID: 656995
Lat: 823136.0
Long: 135764.0
Row ID: 657051
Lat: 823131.0
Long: 135768.0
Row ID: 657080
Lat: 823713.0
Long: 135508.0
Row ID: 657160
Lat: 822478.0
Long: 136052.0
Row ID: 657170
Lat: 821535.0
Long: 134667.0
Row ID: 657173
Lat: 82154.0
Long: 134666.0
Row ID: 657227
Lat: 8242260.0
Long: 136927.0
Row ID: 663523
Lat: 7052250.0
Long: -12.5311
Row ID: 664015
Lat: 824276.0
Long: 136898.0
Row ID: 664021
Lat: 82373.0
Long: 135538.0
Row ID: 664067
Lat: 824304.0
Long: 136912.0
Row ID: 664182
Lat: 823025.0
Long: 13599.0
Row ID: 664997
Lat: 2227630000000000.0
Long: -12.0837
Row ID: 667317
Lat: 823841.0
Long: 135357.0
Row ID: 671201
Lat: 823696.0
Long: 135651.0


In [12]:
#Calculate population within x distance of water point
def calculate_population_within_x_km(row, index, km):
    center_point = GeoPoint(row['#lat_deg'], row['#lon_deg'])
    total_population = 0
    try:
        for point,distance in index.get_nearest_points(center_point, km, 'km'):
            total_population += point.ref
    except:
        print("Invalid data - Record skipped")
    #print("Total population within", km, "kilometers:", int(total_population))
    return total_population

In [13]:
#Calculate # of water points within x distance of water point
def calculate_water_points_within_x_km(row, index, km):
    center_point = GeoPoint(row['#lat_deg'], row['#lon_deg'])
    total_water_points = 0
    total_functioning_water_points = 0
    try:
        for point,distance in index.get_nearest_points(center_point, km, 'km'):
            total_water_points += 1
            if point.ref == 'yes':
                total_functioning_water_points += 1
    except:
        print("Invalid data - Record skipped")
        
    #Adjust for self
    total_water_points -= 1
    if row['#status_id'] == 'yes':
        total_functioning_water_points -= 1
    return (total_water_points, total_functioning_water_points)

In [14]:
#Add 1 km radius population data to each well
tqdm.pandas()
sl_water['1 km population'] = sl_water.progress_apply(lambda row: calculate_population_within_x_km(row, geo_index, 1),
                                                  axis = 1)

 55%|█████▍    | 29968/54864 [38:50:00<15:08:56,  2.19s/it]     

Invalid data - Record skipped


 58%|█████▊    | 31959/54864 [40:58:21<12:12:42,  1.92s/it] 

Invalid data - Record skipped


 72%|███████▏  | 39510/54864 [51:23:40<9:06:00,  2.13s/it]    

...

 91%|█████████ | 49868/54864 [64:04:15<3:20:42,  2.41s/it]  

Invalid data - Record skipped


 98%|█████████▊| 53748/54864 [68:49:15<38:45,  2.08s/it]     

Invalid data - Record skipped


100%|██████████| 54864/54864 [72:23:35<00:00,  4.75s/it]      
/Users/alexjamar/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [15]:
#Add 1 km radius water point data to each water point
tqdm.pandas()
sl_water['1 km water points (total, functioning)'] = sl_water.progress_apply(lambda row: calculate_water_points_within_x_km(row, geo_index_water, 1),
                                                  axis = 1)

 55%|█████▍    | 29972/54864 [19:27<20:43, 20.02it/s]  

Invalid data - Record skipped


 58%|█████▊    | 31962/54864 [20:43<17:58, 21.24it/s]

Invalid data - Record skipped


 72%|███████▏  | 39512/54864 [25:24<12:59, 19.69it/s]

...

 91%|█████████ | 49872/54864 [2:01:21<03:06, 26.76it/s]

Invalid data - Record skipped


 98%|█████████▊| 53751/54864 [2:03:50<00:49, 22.65it/s]

Invalid data - Record skipped


100%|██████████| 54864/54864 [2:04:43<00:00,  7.33it/s]
/Users/alexjamar/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [29]:
sl_water.head(2)

,Row ID,#country_name,#water_source,#water_tech,#status_id,#management,#pay,#installer,#install_year,#status,#source,#adm1,#adm2,#wpdx_id,#report_date,#country_id,#activity_id,#data_lnk,#orig_lnk,photo_lnk,#converted,#created,#updated,#lat_deg,#lon_deg,Location,Count,#fecal_coliform_presence,#fecal_coliform_value,#subjective_quality,1 km population,"1 km water points (total, functioning)"
42474,63115,Sierra Leone,Pump on hand-dug well,India Mark,yes,Community Management,Only if there is a breakdown,OXFAM,2011.0,Yes- functional.,Sierra Leone STATWASH portal,Eastern,Kailahun,wpdx-00063115,03/15/2012 12:00:00 AM +0000,SL,ae734jla,http://www.dropbox.com/s/8sgpa099ebtbg2w/All_W...,NaN,NaN,"#country_id, #status_id, #status, #management,...",10/26/2015 09:24:51 PM +0000,10/26/2015 09:24:51 PM +0000,8.14546,-10.7758,"(8.14546, -10.7758)",1,NaN,NaN,NaN,317.205,"(1, 1)"
42475,63146,Sierra Leone,Pump on hand-dug well,Inkar,yes,Community Management,Yes- regularly,OXFAM,2010.0,Yes- functional.,Sierra Leone STATWASH portal,Eastern,Kailahun,wpdx-00063146,03/14/2012 12:00:00 AM +0000,SL,ahmcg6kg,http://www.dropbox.com/s/8sgpa099ebtbg2w/All_W...,NaN,NaN,"#country_id, #status_id, #status, #management,...",10/26/2015 09:24:51 PM +0000,10/26/2015 09:24:51 PM +0000,8.21998,-10.6945,"(8.21998, -10.6945)",1,NaN,NaN,NaN,309.955,"(4, 3)"


In [30]:
sl_water['1 km population'] = sl_water['1 km population'].apply(lambda x: int(round(x,0)))

/Users/alexjamar/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [32]:
sl_water['1 km total water points'] = sl_water['1 km water points (total, functioning)'].apply(lambda x: str(x).split(',')[0][1:])

/Users/alexjamar/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [33]:
sl_water['1 km functioning water points'] = sl_water['1 km water points (total, functioning)'].apply(lambda x: str(x).split(',')[1][:-1])

/Users/alexjamar/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [34]:
sl_water.head()

,Row ID,#country_name,#water_source,#water_tech,#status_id,#management,#pay,#installer,#install_year,#status,#source,#adm1,#adm2,#wpdx_id,#report_date,#country_id,#activity_id,#data_lnk,#orig_lnk,photo_lnk,#converted,#created,#updated,#lat_deg,#lon_deg,Location,Count,#fecal_coliform_presence,#fecal_coliform_value,#subjective_quality,1 km population,"1 km water points (total, functioning)",1 km total water points,1 km functioning water points
42474,63115,Sierra Leone,Pump on hand-dug well,India Mark,yes,Community Management,Only if there is a breakdown,OXFAM,2011.0,Yes- functional.,Sierra Leone STATWASH portal,Eastern,Kailahun,wpdx-00063115,03/15/2012 12:00:00 AM +0000,SL,ae734jla,http://www.dropbox.com/s/8sgpa099ebtbg2w/All_W...,NaN,NaN,"#country_id, #status_id, #status, #management,...",10/26/2015 09:24:51 PM +0000,10/26/2015 09:24:51 PM +0000,8.14546,-10.7758,"(8.14546, -10.7758)",1,NaN,NaN,NaN,317,"(1, 1)",1,1
42475,63146,Sierra Leone,Pump on hand-dug well,Inkar,yes,Community Management,Yes- regularly,OXFAM,2010.0,Yes- functional.,Sierra Leone STATWASH portal,Eastern,Kailahun,wpdx-00063146,03/14/2012 12:00:00 AM +0000,SL,ahmcg6kg,http://www.dropbox.com/s/8sgpa099ebtbg2w/All_W...,NaN,NaN,"#country_id, #status_id, #status, #management,...",10/26/2015 09:24:51 PM +0000,10/26/2015 09:24:51 PM +0000,8.21998,-10.6945,"(8.21998, -10.6945)",1,NaN,NaN,NaN,310,"(4, 3)",4,3
42476,63116,Sierra Leone,Pump on hand-dug well,India Mark,yes,Community Management,No- water is free,Unknown,2011.0,Yes- functional.,Sierra Leone STATWASH portal,Eastern,Kailahun,wpdx-00063116,03/14/2012 12:00:00 AM +0000,SL,alqbuvky,http://www.dropbox.com/s/8sgpa099ebtbg2w/All_W...,NaN,NaN,"#country_id, #status_id, #status, #management,...",10/26/2015 09:24:51 PM +0000,10/26/2015 09:24:51 PM +0000,8.30947,-10.6979,"(8.30947, -10.6979)",1,NaN,NaN,NaN,253,"(0, 0)",0,0
42477,63117,Sierra Leone,Pump on hand-dug well,India Mark,yes,Community Management,Yes- regularly,OXFAM,2011.0,Yes- functional.,Sierra Leone STATWASH portal,Eastern,Kailahun,wpdx-00063117,03/08/2012 12:00:00 AM +0000,SL,aln4y85f,http://www.dropbox.com/s/8sgpa099ebtbg2w/All_W...,NaN,NaN,"#country_id, #status_id, #status, #management,...",10/26/2015 09:24:51 PM +0000,10/26/2015 09:24:51 PM +0000,8.30754,-10.7395,"(8.30754, -10.7395)",1,NaN,NaN,NaN,317,"(1, 1)",1,1
42478,63118,Sierra Leone,Pump on hand-dug well,India Mark,no,Community Management,No- water is free,cord sl,2011.0,No- broken down. Pump damaged,Sierra Leone STATWASH portal,Eastern,Kailahun,wpdx-00063118,03/09/2012 12:00:00 AM +0000,SL,alafpbam,http://www.dropbox.com/s/8sgpa099ebtbg2w/All_W...,NaN,NaN,"#country_id, #status_id, #status, #management,...",10/26/2015 09:24:51 PM +0000,10/26/2015 09:24:51 PM +0000,8.29985,-10.7054,"(8.29985, -10.7054)",1,NaN,NaN,NaN,22,"(4, 3)",4,3


In [35]:
sl_abridged = sl_water[['Row ID', '#country_name', '#status_id', '#lat_deg', '#lon_deg', '1 km population',
                        '1 km total water points', '1 km functioning water points']]

In [36]:
sl_abridged.head()

,Row ID,#country_name,#status_id,#lat_deg,#lon_deg,1 km population,1 km total water points,1 km functioning water points
42474,63115,Sierra Leone,yes,8.14546,-10.7758,317,1,1
42475,63146,Sierra Leone,yes,8.21998,-10.6945,310,4,3
42476,63116,Sierra Leone,yes,8.30947,-10.6979,253,0,0
42477,63117,Sierra Leone,yes,8.30754,-10.7395,317,1,1
42478,63118,Sierra Leone,no,8.29985,-10.7054,22,4,3


In [38]:
sl_abridged.to_csv("Abridged_Sierra_Leone_Water_and_Population_Data.csv", index = False)